# Del 07: Python Wheels

https://realpython.com/python-wheels/

Python .whl files, or wheels, are a little-discussed part of Python, but they’ve been a boon to the installation process for Python packages. If you’ve installed a Python package using pip, then chances are that a wheel has made the installation faster and more efficient.

Wheels are a component of the Python ecosystem that helps to make package installs just work. They allow for faster installations and more stability in the package distribution process. In this tutorial, you’ll dive into what wheels are, what good they serve, and how they’ve gained traction and made Python even more of a joy to work with.

## Setup

Pripravimo okolje:

    mkdir Del_07_Python_Wheels
    cd Del_07_Python_Wheels/
    pyenv local 3.9.0
    python --version

Ustvarimo virtualno okolje:
    
    python -m venv .venv
    source .venv/bin/activate

To follow along, activate a virtual environment and make sure you have the latest versions of pip, wheel, and setuptools installed:

    python -m pip install -U pip wheel setuptools

> Odpremo VS Code v tej mapi -> izberemo iterpreter

That’s all you need to experiment with installing and building wheels!

## Python Packaging Made Better: An Intro to Python Wheels

Before you learn how to package a project into a wheel, it helps to know what using one looks like from the user’s side. It may sound backward, but a good way to learn how wheels work is to start by installing something that isn’t a wheel.

You can start this experiment by installing a Python package into your environment just as you might normally do. In this case, install uWSGI version 2.0.x:

    python -m pip install 'uwsgi==2.0.*'

    Collecting uwsgi==2.0.*
      Downloading uWSGI-2.0.19.1.tar.gz (803 kB)
         |████████████████████████████████| 803 kB 19.8 MB/s
    Building wheels for collected packages: uwsgi
      Building wheel for uwsgi (setup.py) ... done
      Created wheel for uwsgi: filename=uWSGI-2.0.19.1-cp39-cp39-linux_x86_64.whl size=7565644 sha256=4932e8a8155e523edc14a85c35b6de23c356fccb9dac553ca0d7d7a499750aca
      Stored in directory: /home/leons/.cache/pip/wheels/17/66/cf/112237fefe0b8011e4a481305ba799b696ece8590219bd527f
    Successfully built uwsgi
    Installing collected packages: uwsgi
    Successfully installed uwsgi-2.0.19.1

To fully install uWSGI, pip progresses through several distinct steps:
1. On line 3, it downloads a TAR file (tarball) named uwsgi-2.0.18.tar.gz that’s been compressed with gzip.
2. On line 6, it takes the tarball and builds a .whl file through a call to setup.py.
3. On line 7, it labels the wheel uWSGI-2.0.18-cp38-cp38-macosx_10_15_x86_64.whl.
4. On line 10, it installs the actual package after having built the wheel.

The tar.gz tarball that pip retrieves is a source distribution, or `sdist`, rather than a wheel. In some ways, a sdist is the opposite of a wheel.

> <p><strong>Note</strong>: If you see an error with the uWSGI installation, you may need to <a href="https://uwsgi-docs.readthedocs.io/en/latest/Install.html#installing-from-source">install the Python development headers</a>.</p>

A source distribution contains source code. That includes not only Python code but also the source code of any extension modules (usually in C or C++) bundled with the package. With source distributions, extension modules are compiled on the user’s side rather than the developer’s.

Source distributions also contain a bundle of metadata sitting in a directory called <package-name>.egg-info. This metadata helps with building and installing the package, but user’s don’t really need to do anything with it.

From the developer’s perspective, a source distribution is what gets created when you run the following command:

> `python setup.py sdist`

Now try installing a different package, chardet:

    python -m pip install 'chardet==3.*'

    Collecting chardet==3.*
      Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
    Installing collected packages: chardet
    Successfully installed chardet-3.0.4

You can see a noticeably different output than the uWSGI install.

Installing chardet downloads a .whl file directly from PyPI. The wheel name chardet-3.0.4-py2.py3-none-any.whl follows a specific naming convention that you’ll see later. What’s more important from the user’s perspective is that there’s no build stage when pip finds a compatible wheel on PyPI.

From the developer’s side, a wheel is the result of running the following command:

> `python setup.py bdist_wheel`

Why does uWSGI hand you a source distribution while chardet provides a wheel? You can see the reason for this by taking a look at each project’s page on PyPI and navigating to the Download files area. This section will show you what pip actually sees on the PyPI index server:

<ul>
<li><strong>uWSGI</strong> <a href="https://pypi.org/project/uWSGI/2.0.18/#files">provides only a source distribution</a> (<code>uwsgi-2.0.18.tar.gz</code>) for reasons related to the complexity of the project.</li>
<li><strong>chardet</strong> <a href="https://pypi.org/project/chardet/3.0.4/#files">provides both a wheel and a source distribution</a>, but <code>pip</code> will prefer the wheel <em>if</em> it’s compatible with your system. You’ll see how that compatibility is determined later on.</li>
</ul>

Another example of the compatibility check used for wheel installation is psycopg2, which provides a wide set of wheels for Windows but doesn’t provide any for Linux or macOS clients. This means that pip install psycopg2 could fetch a wheel or a source distribution depending on your specific setup.

To avoid these types of compatibility issues, some packages offer multiple wheels, with each wheel geared toward a specific Python implementation and underlying operating system.

So far, you’ve seen some of the visible distinctions between a wheel and sdist, but what matters more is the impact those differences have on the installation process.

### Wheels Make Things Go Fast

Above, you saw a comparison of an installation that fetches a prebuilt wheel and one that downloads a sdist. Wheels make the end-to-end installation of Python packages faster for two reasons:

1. All else being equal, wheels are typically smaller in size than source distributions, meaning they can move faster across a network.
2. Installing from wheels directly avoids the intermediate step of building packages off of the source distribution.

It’s almost guaranteed that the chardet install occurred in a fraction of the time required for uWSGI. However, that’s arguably an unfair apples-to-oranges comparison since chardet is a significantly smaller and less complex package. With a different command, you can create a more direct comparison that will demonstrate just how much of a difference wheels make.

You can make pip ignore its inclination towards wheels by passing the --no-binary option:

    time python -m pip install \
          --no-cache-dir \
          --force-reinstall \
          --no-binary=:all: \
          cryptography

    real    0m35.764s
    user    0m34.096s
    sys     0m1.481s

This command times the installation of the cryptography package, telling pip to use a source distribution even if a suitable wheel is available. Including :all: makes the rule apply to cryptography and all of its dependencies.

On my machine, this takes around thirty-two seconds from start to finish. Not only does the install take a long time, but building cryptography also requires that you have the OpenSSL development headers present and available to Python.

> <p><strong>Note</strong>: With <code>--no-binary</code>, you may very well see an error about missing header files required for the <code>cryptography</code> install, which is part of what can make using source distributions frustrating. If so, the <a href="https://cryptography.io/en/latest/installation/#building-cryptography-on-linux">installation section</a> of the <code>cryptography</code> docs advises on which libraries and header files you’ll need for a particular operating system.</p>

Now you can reinstall cryptography, but this time make sure that pip uses wheels from PyPI. Because pip will prefer a wheel, this is similar to just calling pip install with no arguments at all. But in this case, you can make the intent explicit by requiring a wheel with --only-binary:

    time python -m pip install \
          --no-cache-dir \
          --force-reinstall \
          --only-binary=cryptography \
          cryptography

    real    0m1.859s
    user    0m1.599s
    sys     0m0.153s

This option takes just over four seconds, or one-eighth the time that it took when using only source distributions for cryptography and its dependencies.

### What Is a Python Wheel?

A Python .whl file is essentially a ZIP (.zip) archive with a specially crafted filename that tells installers what Python versions and platforms the wheel will support.

A wheel is a type of built distribution. In this case, built means that the wheel comes in a ready-to-install format and allows you to skip the build stage required with source distributions.

> Note: It’s worth mentioning that despite the use of the term built, a wheel doesn’t contain .pyc files, or compiled Python bytecode.

A wheel filename is broken down into parts separated by hyphens:

    {dist}-{version}(-{build})?-{python}-{abi}-{platform}.whl

Each section in {brackets} is a tag, or a component of the wheel name that carries some meaning about what the wheel contains and where the wheel will or will not work.

Here’s an illustrative example using a cryptography wheel:

    cryptography-2.9.2-cp35-abi3-macosx_10_9_x86_64.whl

cryptography distributes multiple wheels. Each wheel is a platform wheel, meaning it supports only specific combinations of Python versions, Python ABIs, operating systems, and machine architectures. You can break down the naming convention into parts:

<ul>
<li>
<p><strong><code>cryptography</code></strong> is the package name.</p>
</li>
<li>
<p><strong><code>2.9.2</code></strong> is the package version of <code>cryptography</code>. A version is a <a href="https://www.python.org/dev/peps/pep-0440/">PEP 440</a>-compliant string such as <code>2.9.2</code>, <code>3.4</code>, or <code>3.9.0.a3</code>.</p>
</li>
<li>
<p><strong><code>cp35</code></strong> is the <a href="https://www.python.org/dev/peps/pep-0425/#python-tag">Python tag</a> and denotes the Python implementation and version that the wheel demands. The <code>cp</code> stands for <a href="https://github.com/python/cpython">CPython</a>, the reference implementation of Python, while the <code>35</code> denotes Python <a href="https://docs.python.org/3/whatsnew/3.5.html">3.5</a>. This wheel wouldn’t be compatible with <a href="https://www.jython.org/">Jython</a>, for instance.</p>
</li>
<li>
<p><strong><code>abi3</code></strong> is the ABI tag. ABI stands for <a href="https://docs.python.org/3/c-api/stable.html">application binary interface</a>. You don’t really need to worry about what it entails, but <code>abi3</code> is a separate version for the binary compatibility of the Python C API.</p>
</li>
<li>
<p><strong><code>macosx_10_9_x86_64</code></strong> is the platform tag, which happens to be quite a mouthful. In this case it can be broken down further into sub-parts:</p>
<ul>
<li><strong><code>macosx</code></strong> is the <a href="https://en.wikipedia.org/wiki/MacOS">macOS</a> operating system.</li>
<li><strong><code>10_9</code></strong> is the macOS developer tools SDK version used to compile the Python that in turn built this wheel.</li>
<li><strong><code>x86_64</code></strong> is a reference to x86-64 instruction set architecture.</li>
</ul>
</li>
</ul>

The final component isn’t technically a tag but rather the standard .whl file extension. Combined, the above components indicate the target machine that this cryptography wheel is designed for.

Now let’s turn to a different example. Here’s what you saw in the above case for chardet:



    chardet-3.0.4-py2.py3-none-any.whl

You can break this down into its tags:

<ul>
<li><strong><code>chardet</code></strong> is the package name.</li>
<li><strong><code>3.0.4</code></strong> is the package version of chardet.</li>
<li><strong><code>py2.py3</code></strong> is the Python tag, meaning the wheel supports Python 2 and 3 with any Python implementation.</li>
<li><strong><code>none</code></strong> is the ABI tag, meaning the ABI isn’t a factor.</li>
<li><strong><code>any</code></strong> is the platform. This wheel will work on virtually any platform.</li>
</ul>

The py2.py3-none-any.whl segment of the wheel name is common. This is a universal wheel that will install with Python 2 or 3 on any platform with any ABI. If the wheel ends in none-any.whl, then it’s very likely a pure-Python package that doesn’t care about a specific Python ABI or CPU architecture.

### Advantages of Python Wheels

Here’s a testament to wheels from the Python Packaging Authority (PyPA):

> Not all developers have the right tools or experiences to build these components written in these compiled languages, so Python created the wheel, a package format designed to ship libraries with compiled artifacts. In fact, Python’s package installer, pip, always prefers wheels because installation is always faster, so even pure-Python packages work better with wheels. (Source)

A fuller description is that wheels benefit both users and maintainers of Python packages alike in a handful of ways:

<ul>
<li>
<p><strong>Wheels install faster</strong> than source distributions for both pure-Python packages and <a href="https://realpython.com/build-python-c-extension-module/">extension modules</a>.</p>
</li>
<li>
<p><strong>Wheels are smaller</strong> than source distributions. For example, the <a href="https://pypi.org/project/six/#files"><code>six</code></a> wheel is about <a href="https://pypi.org/project/six/#files">one-third the size</a> of the corresponding source distribution. This differential becomes even more important when you consider that a <code>pip install</code> for a single package may actually kick off downloading a chain of dependencies.</p>
</li>
<li>
<p><strong>Wheels cut <code>setup.py</code> execution out of the equation.</strong> Installing from a source distribution runs <em>whatever</em> is contained in that project’s <code>setup.py</code>. As pointed out by <a href="https://www.python.org/dev/peps/pep-0427/#rationale">PEP 427</a>, this amounts to arbitrary code execution. Wheels avoid this altogether.</p>
</li>
<li>
<p><strong>There’s no need for a compiler</strong> to install wheels that contain compiled extension modules. The extension module comes included with the wheel targeting a specific platform and Python version.</p>
</li>
<li>
<p><strong><code>pip</code> automatically generates <code>.pyc</code> files</strong> in the wheel that match the right Python interpreter.</p>
</li>
<li>
<p><strong>Wheels provide consistency</strong> by cutting many of the variables involved in installing a package out of the equation.</p>
</li>
</ul>

> `.pyc file`: They contain byte code, which is what the Python interpreter compiles the source to. This code is then executed by Python's virtual machine.

### Telling pip What to Download

It’s possible to exert fine-grained control over pip and tell it which format to prefer or avoid. You can use the --only-binary and --no-binary options to do this. You saw these used in an earlier section on installing the cryptography package, but it’s worth taking a closer look at what they do:

    pushd "$(mktemp -d)"
    python -m pip download --only-binary :all: --dest . --no-cache six

In this example, you change to a temporary directory to store the download with pushd "$(mktemp -d)". You use pip download rather than pip install so that you can inspect the resulting wheel, but you can replace download with install while keeping the same set of options.

You download the six module with several flags:

<ul>
<li><strong><code>--only-binary :all:</code></strong> tells <code>pip</code> to constrain itself to using wheels and ignore source distributions. Without this option, <code>pip</code> will only <em>prefer</em> wheels but will fall back to source distributions in some scenarios.</li>
<li><strong><code>--dest .</code></strong> tells <code>pip</code> to download <code>six</code> to the current directory.</li>
<li><strong><code>--no-cache</code></strong> tells <code>pip</code> not to look in its local download cache. You use this option just to illustrate a live download from PyPI since it’s likely you do have a <code>six</code> cache somewhere.</li>
</ul>

I mentioned earlier that a wheel file is essentially a .zip archive. You can take this statement literally and treat wheels as such. For instance, if you want to view a wheel’s contents, you can use unzip:

    sudo apt install unzip
    unzip -l six*.whl

six is a special case: it’s actually a single Python module rather than a complete package. Wheel files can also be significantly more complex, as you’ll see later on.

In contrast to --only-binary, you can use --no-binary to do the opposite:

    python -m pip download --no-binary :all: --dest . --no-cache six
    popd

The only change in this example is the switch to --no-binary :all:. This tells pip to ignore wheels even if they’re available and instead download a source distribution.

When might --no-binary be useful? Here are a few cases:

<ul>
<li>
<p><strong>The corresponding wheel is broken.</strong> This is an irony of wheels. They’re designed to make things break less often, but in some cases a wheel can be misconfigured. In this case, downloading and building the source distribution for yourself may be a working alternative.</p>
</li>
<li>
<p><strong>You want to apply a small change or <a href="https://en.wikipedia.org/wiki/Patch_%28Unix%29">patch file</a></strong> to the project and then install it. This is an alternative to cloning the project from its <a href="https://realpython.com/python-git-github-intro/#version-control">version control system</a> URL.</p>
</li>
</ul>

You can also use the flags described above with pip install. Additionally, instead of :all:, which will apply the --only-binary rule not just to the package you’re installing but to all of its dependencies, you can pass --only-binary and --no-binary a list of specific packages to apply that rule to.

Here are a few examples for installing the URL library yarl. It contains Cython code and depends on multidict, which contains pure C code. There are several options to strictly use or strictly ignore wheels for yarl and its dependencies:

    $ # Install `yarl` and use only wheels for yarl and all dependencies
    $ python -m pip install --only-binary :all: yarl

    $ # Install `yarl` and use wheels only for the `multidict` dependency
    $ python -m pip install --only-binary multidict yarl

    $ # Install `yarl` and don't use wheels for yarl or any dependencies
    $ python -m pip install --no-binary :all: yarl

    $ # Install `yarl` and don't use wheels for the `multidict` dependency
    $ python -m pip install --no-binary multidict yarl

In this section, you got a glimpse of how to fine-tune the distribution types that pip install will use. While a regular pip install should work with no options, it’s helpful to know these options for special cases.

## Build Your Wheels

The title of this tutorial asks, “Why Should You Care?” As a developer, if you plan to distribute a Python package to the community, then you should care immensely about distributing wheels for your project because they make the installation process cleaner and less complex for end users.

The more target platforms that you can support with compatible wheels, the fewer GitHub issues you’ll see titled something like “Installation broken on Platform XYZ.” Distributing wheels for your Python package makes it objectively less likely that users of the package will encounter issues during installation.

### Different Types of Wheels

As touched on throughout this tutorial, there are several different variations of wheels, and the wheel’s type is reflected in its filename:

<ul>
<li>
<p>A <strong>universal wheel</strong> contains <code>py2.py3-none-any.whl</code>. It supports both Python 2 and Python 3 on any OS and platform. The majority of wheels listed on the <a href="https://pythonwheels.com/">Python Wheels</a> website are universal wheels.</p>
</li>
<li>
<p>A <strong>pure-Python wheel</strong> contains either <code>py3-none-any.whl</code> or <code>py2.none-any.whl</code>. It supports either Python 3 or Python 2, but not both. It’s otherwise the same as a universal wheel, but it’ll be labeled with either <code>py2</code> or <code>py3</code> rather than the <code>py2.py3</code> label.</p>
</li>
<li>
<p>A <strong>platform wheel</strong> supports a specific Python version and platform. It contains segments indicating a specific Python version, ABI, operating system, or architecture.</p>
</li>
</ul>

The differences between wheel types are determined by which version(s) of Python they support and whether they target a specific platform. Here’s a condensed summary of the differences between wheel variations:

<table class="table table-hover">
<thead>
<tr>
<th>Wheel Type</th>
<th>Supports Python 2 and 3</th>
<th>Supports Every ABI, OS, and Platform</th>
</tr>
</thead>
<tbody>
<tr>
<td>Universal</td>
<td>✓</td>
<td>✓</td>
</tr>
<tr>
<td>Pure-Python</td>
<td></td>
<td>✓</td>
</tr>
<tr>
<td>Platform</td>
<td></td>
<td></td>
</tr>
</tbody>
</table>

As you’ll see next, you can build universal wheels and pure-Python wheels with relatively little setup, but platform wheels may require a few additional steps.

### Building a Pure-Python Wheel

You can build a pure-Python wheel or a universal wheel for any project using setuptools with just a single command:

    python setup.py sdist bdist_wheel

This will create both a source distribution (sdist) and a wheel (bdist_wheel). By default, both will be placed in dist/ under the current directory. To see for yourself, you can build a wheel for HTTPie, a command-line HTTP client written in Python, alongside a sdist.

Here’s the result of building both types of distributions for the HTTPie package:

    git clone -q https://github.com/httpie/httpie.git

    cd httpie
    python setup.py -q sdist bdist_wheel
    ls -1 dist/

That’s all it takes. You clone the project, move into its root directory, and then call python setup.py sdist bdist_wheel. You can see that dist/ contains both a wheel and a source distribution.

The resulting distributions get put in dist/ by default, but you can change that with the -d/--dist-dir option. 

### Specifying a Universal Wheel

A universal wheel is a wheel for a pure-Python project that supports both Python 2 and 3. There are multiple ways to tell setuptools and distutils that a wheel should be universal.

Option 1 is to specify the option in your project’s setup.cfg file:



    [bdist_wheel]
    universal = 1

Option 2 is to pass the aptly named --universal flag at the command line:

    python setup.py bdist_wheel --universal

Option 3 is to tell setup() itself about the flag using its options parameter:

    # setup.py
    from setuptools import setup

    setup(
        # ....
        options={"bdist_wheel": {"universal": True}}
        # ....
    )

While any of these three options should work, the first two are used most frequently. You can see an example of this in the chardet setup configuration.

The resulting wheel will be equivalent no matter which option you choose. The choice largely comes down to developer preference and which workflow is best for you.



## Wheel vs Egg

<div class="section" id="wheel-vs-egg">
<p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> and <a class="reference internal" href="../../glossary/#term-Egg"><span class="xref std std-term">Egg</span></a> are both packaging formats that aim to support the
use case of needing an install artifact that doesn’t require building or
compilation, which can be costly in testing and production workflows.</p>
<p>The <a class="reference internal" href="../../glossary/#term-Egg"><span class="xref std std-term">Egg</span></a> format was introduced by <a class="reference internal" href="../../key_projects/#setuptools"><span class="std std-ref">setuptools</span></a> in 2004, whereas the
<a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> format was introduced by <span class="target" id="index-0"></span><a class="pep reference external" href="https://www.python.org/dev/peps/pep-0427"><strong>PEP 427</strong></a> in 2012.</p>
<p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> is currently considered the standard for <a class="reference internal" href="../../glossary/#term-Built-Distribution"><span class="xref std std-term">built</span></a> and <a class="reference internal" href="../../glossary/#term-Binary-Distribution"><span class="xref std std-term">binary</span></a> packaging for Python.</p>
<p>Here’s a breakdown of the important differences between <a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> and <a class="reference internal" href="../../glossary/#term-Egg"><span class="xref std std-term">Egg</span></a>.</p>
<ul class="simple">
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> has an <span class="target" id="index-1"></span><a class="pep reference external" href="https://www.python.org/dev/peps/pep-0427"><strong>official PEP</strong></a>. <a class="reference internal" href="../../glossary/#term-Egg"><span class="xref std std-term">Egg</span></a> did not.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> is a <a class="reference internal" href="../../glossary/#term-Distribution-Package"><span class="xref std std-term">distribution</span></a> format, i.e a packaging
format. <a class="footnote-reference brackets" href="#id3" id="id2">1</a> <a class="reference internal" href="../../glossary/#term-Egg"><span class="xref std std-term">Egg</span></a> was both a distribution format and a runtime
installation format (if left zipped), and was designed to be importable.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> archives do not include .pyc files. Therefore, when the
distribution only contains Python files (i.e. no compiled extensions), and is
compatible with Python 2 and 3, it’s possible for a wheel to be “universal”,
similar to an <a class="reference internal" href="../../glossary/#term-Source-Distribution-or-sdist"><span class="xref std std-term">sdist</span></a>.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> uses <span class="target" id="index-2"></span><a class="pep reference external" href="https://www.python.org/dev/peps/pep-0376"><strong>PEP376-compliant</strong></a> <code class="docutils literal notranslate"><span class="pre">.dist-info</span></code>
directories. Egg used <code class="docutils literal notranslate"><span class="pre">.egg-info</span></code>.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> has a <span class="target" id="index-3"></span><a class="pep reference external" href="https://www.python.org/dev/peps/pep-0425"><strong>richer file naming convention</strong></a>. A single
wheel archive can indicate its compatibility with a number of Python language
versions and implementations, ABIs, and system architectures.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> is versioned. Every wheel file contains the version of the wheel
specification and the implementation that packaged it.</p></li>
<li><p><a class="reference internal" href="../../glossary/#term-Wheel"><span class="xref std std-term">Wheel</span></a> is internally organized by <a class="reference external" href="http://docs.python.org/2/library/sysconfig.html#installation-paths">sysconfig path type</a>,
therefore making it easier to convert to other formats.</p></li>
</ul>

</div>